In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog
from DEA_Solver import DEA_CCR

datas=pd.read_csv("datas.csv", sep=";", index_col=False)
xi, xo= datas.values[:,1:2].astype(np.float32), datas.values[:,2:].astype(np.float32)
datas

,attr,price ($),storage (GB),camera (MP),screen (mm)
0,mobile1,250,16,12,4.0
1,mobile2,225,16,8,5.0
2,mobile3,300,32,16,4.5
3,mobile4,275,32,8,4.0


In [2]:
datas_values=datas.values[:,1:].astype(np.float32)
datas_values

array([[250. ,  16. ,  12. ,   4. ],
       [225. ,  16. ,   8. ,   5. ],
       [300. ,  32. ,  16. ,   4.5],
       [275. ,  32. ,   8. ,   4. ]], dtype=float32)

<h3 style="color:yellow">Normalization</h3>

In [3]:
datas_norm=np.sqrt(np.sum(datas_values**2, axis=0, keepdims=True))
datas_values/datas_norm

array([[0.47351375, 0.31622776, 0.52223295, 0.4551041 ],
       [0.42616236, 0.31622776, 0.34815532, 0.56888014],
       [0.5682165 , 0.6324555 , 0.69631064, 0.5119921 ],
       [0.52086514, 0.6324555 , 0.34815532, 0.4551041 ]], dtype=float32)

In [4]:
res=DEA_CCR(xi, xo)

In [8]:
a=res.solve()
a

array([[250.    ,  16.    ,  12.    ,   4.    ,   0.9681],
       [225.    ,  16.    ,   8.    ,   5.    ,   1.    ],
       [300.    ,  32.    ,  16.    ,   4.5   ,   1.    ],
       [275.    ,  32.    ,   8.    ,   4.    ,   1.    ]])

<h2 style="color:yellow">Lingprog</h2>
O lingprog resolve o problema de minimização da forma:

$\text{min}\quad C^Tx$\
st.\
$ A_{ub} x \leq b_{ub}$\
$ A_{eq} x = b_{eq}$\
$ l\leq x \leq u $

Onde:
- $C^T$ --- obj (objective function)
- $A_{ub}$ --- lhs_ineq (left hand side inequality constraint)
- $b_{ub}$ --- rhs_ineq (right hand side inequality constraint)
- $A_{eq}$ --- lhs_eq (left hand side equality constraint)
- $b_{eq}$ --- rhs_eq (right hand side equality constraint) 

Todas as restrições com sinal de maior ou igual ($\geq$) devem ser convertidas para menor ou igual ($\leq$) multiplicando por -1. 

Para problema de maximizacao basta multiplicar a função objetivo por -1.

Outra coisa que deve ser levada em consideração são as variáveis de decisão de não negatividade. Para definir isso, crie tuplas para cada variável. Por exemplo, se uma variável $x_1$ tem que ser maior ou igual à zero, use `x_1=(0,None)`. Pode trocar o `None` por `np.inf` ou `float('inf')`. Se a variável $x_1$ é livre, pode escrever `x_1=(None,None)`. Em suma, para todas variáveis `x`, deverá indicar o lower e o upper `x=(l,u)`, mesmo que ambas sejam `None`.

A sinatxe para lingprog é:

```python
linprog(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None, bounds=(0, None))
```

Na variável `bounds` deverá mandar uma lista com todas variáveis definidas. Exemplo: `bounds=[x_0, x_1, x_2]`.


Documentação:
* [Documentação do lingprog no Scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html)
* [Exemplos com explicação](https://realpython.com/linear-programming-python/)
* [Outros Exemplos](https://www.cuemath.com/algebra/linear-programming/)


In [11]:
# minimize
c0=np.array([-1, 4])
st0=np.array([
    [-3, 1, 6],#<=
    [-1, -2, -4]#>=
    ])
bs0_x0=(None, None)
bs0_x1=(-3, None)

# Solucao 
A_ub0=st0[:,:-1]*[[1],[-1]] 
b_ub0=st0[:,-1]*[1,-1]
#multiplicamos por -1 a segunda restricao para mudar a direcao da desigualdade
# x1=10 e x2=-3 z=-22 sao as solucoes

# maximize
c1=np.array([40, 30])
st1=np.array([
    [1, 1, 12],#<=
    [2, 1, 16] #<=
    ])
bs1_x0=(0, None)
bs1_x1=(0, None)

# Solucao
c1=c1*(-1) #multiplicamos por -1 a funcao objetivo para converter para minimizacao
A_ub1=st1[:,:-1]
b_ub1=st1[:,-1]
# x1=4 e x2=8 z=400 sao as solucoes


# Minimize
c2=np.array([5,4])
st2=np.array([
    [4,1,40],#>=
    [2,3,90] #>=
             ])
bs2_x0=(0, None)
bs2_x1=(0, None)

# Solucao
st2=st2*(-1) # Multiplicar st por -1 para inverter o sinal de desigualdade
A_ub2=st2[:,:-1]
b_ub2=st2[:,-1]
# x1=3 e x2=28 z=127 sao as solucoes

# minimize
c3=np.array([1,2,3])
st3=np.array([
    [1,1,1,12],#<=
    [2,1,3,18]#<=
])
bs3_x0=(0, None)
bs3_x1=(0, None)
bs3_x2=(0, None)

# Solucao
A_ub3=st3[:,:-1]
b_ub3=st3[:,-1]

# x1=0 e x2=9 x3=3 z=27 sao as solucoes. Mas acho estao resolver maximizacao, fizeram c3=c3*(-1)

In [12]:
# Exemplo 0 
linprog(c0, A_ub0, b_ub0, bounds=[bs0_x0, bs0_x1])

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -22.0
              x: [ 1.000e+01 -3.000e+00]
            nit: 0
          lower:  residual: [       inf  0.000e+00]
                 marginals: [ 0.000e+00  6.000e+00]
          upper:  residual: [       inf        inf]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 3.900e+01  0.000e+00]
                 marginals: [-0.000e+00 -1.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

In [13]:
# Exemplo 1 
linprog(c1, A_ub1, b_ub1, bounds=[bs1_x0, bs1_x1])

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -400.0
              x: [ 4.000e+00  8.000e+00]
            nit: 2
          lower:  residual: [ 4.000e+00  8.000e+00]
                 marginals: [ 0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00]
                 marginals: [-2.000e+01 -1.000e+01]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

In [14]:
# Exemplo 2 
linprog(c2, A_ub2, b_ub2, bounds=[bs2_x0, bs2_x1])

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 127.0
              x: [ 3.000e+00  2.800e+01]
            nit: 2
          lower:  residual: [ 3.000e+00  2.800e+01]
                 marginals: [ 0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00]
                 marginals: [-7.000e-01 -1.100e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

In [15]:
# Exemplo 3 
linprog(c3, A_ub3, b_ub3, bounds=[bs3_x0, bs3_x1, bs3_x2])

       message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
       success: True
        status: 0
           fun: 0.0
             x: [ 0.000e+00  0.000e+00  0.000e+00]
           nit: 0
         lower:  residual: [ 0.000e+00  0.000e+00  0.000e+00]
                marginals: [ 1.000e+00  2.000e+00  3.000e+00]
         upper:  residual: [       inf        inf        inf]
                marginals: [ 0.000e+00  0.000e+00  0.000e+00]
         eqlin:  residual: []
                marginals: []
       ineqlin:  residual: [ 1.200e+01  1.800e+01]
                marginals: [-0.000e+00 -0.000e+00]